***content based movie recommendation system***


In [44]:
import pandas as pd 
import numpy as np

In [45]:
df=pd.read_csv("/content/C__Users_Nabeel-pc_Desktop_movies_dataframe_grouplens.csv")
df.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [46]:
df.shape

(3883, 3)

In [47]:
df_users=pd.read_csv("/content/users.csv")
df_users.head()

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,2460
4,5,M,25,20,55455


In [48]:
df_users.shape

(6040, 5)

In [49]:
df_ratings=pd.read_csv("/content/ratings.csv")
df_ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [50]:
df_ratings.shape

(1000209, 4)

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3883 entries, 0 to 3882
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   MovieID  3883 non-null   int64 
 1   Title    3883 non-null   object
 2   Genres   3883 non-null   object
dtypes: int64(1), object(2)
memory usage: 91.1+ KB


In [52]:
#extract year from a title
df['year']=df['Title'].str.extract('.*\((.*)\).*',expand=False)
df.head()

,MovieID,Title,Genres,year
0,1,Toy Story (1995),Animation|Children's|Comedy,1995
1,2,Jumanji (1995),Adventure|Children's|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


In [53]:
#replacing '|' with whitespace in 'Genres' column
df['Genres']=df['Genres'].str.replace('|'," ")

In [54]:
df['Genres'].head()

0     Animation Children's Comedy
1    Adventure Children's Fantasy
2                  Comedy Romance
3                    Comedy Drama
4                          Comedy
Name: Genres, dtype: object

***code for content based Recommendation***

In [55]:
df.head(1)['Genres']

0    Animation Children's Comedy
Name: Genres, dtype: object

***TfidfVectorizer function from scikit-learn, which transforms text to feature vectors that can be used as input to estimator.***

In [56]:
#fitting the TF-IDF on 'Grnres'
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(min_df=0,max_features=None,analyzer='word',token_pattern='\w{1,}', ngram_range=(1, 2), stop_words='english')
tfidf_matrix = tf.fit_transform(df['Genres'])

In [57]:
tfidf_matrix

<3883x129 sparse matrix of type '<class 'numpy.float64'>'
	with 10075 stored elements in Compressed Sparse Row format>

In [58]:
tfidf_matrix.shape

(3883, 129)

In [59]:
#The sigmoid is basically responsible for transforming our input features from 0 to 1 and its formula is 1/(1+e(power-1))
from sklearn.metrics.pairwise import sigmoid_kernel
#compute the sigmoid kernel
cosine_sim = sigmoid_kernel(tfidf_matrix, tfidf_matrix)

In [60]:
cosine_sim[0]

array([0.7648306 , 0.76266997, 0.76184872, ..., 0.76159416, 0.76159416,
       0.76159416])

In [61]:
#Reverse mapping of indices and movie titles
titles = df['Title']
indices = pd.Series(df.index, index=df['Title'].drop_duplicates())

In [62]:
indices[: 10]

Title
Toy Story (1995)                      0
Jumanji (1995)                        1
Grumpier Old Men (1995)               2
Waiting to Exhale (1995)              3
Father of the Bride Part II (1995)    4
Heat (1995)                           5
Sabrina (1995)                        6
Tom and Huck (1995)                   7
Sudden Death (1995)                   8
GoldenEye (1995)                      9
dtype: int64

In [63]:
def recomendation(title,cosine_sim=cosine_sim):
  idx = indices[title]
  sim_scores = list(enumerate(cosine_sim[idx]))
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  sim_scores = sim_scores[1:21]
  movie_indices = [i[0] for i in sim_scores]
  return df['Title'].iloc[movie_indices]

In [66]:
#testing our content based recoendation with Tom and Huck (1995
recomendation('Tom and Huck (1995)')           

144                   Amazing Panda Adventure, The (1995)
156                                         Casper (1995)
235     Far From Home: The Adventures of Yellow Dog (1...
388            Secret Adventures of Tom Thumb, The (1993)
417                                   Black Beauty (1994)
480                                         Lassie (1994)
573                                          Andre (1994)
605       Homeward Bound II: Lost in San Francisco (1996)
702                                        Flipper (1996)
744      Gold Diggers: The Secret of Bear Mountain (1995)
798                                         Alaska (1996)
817                   Adventures of Pinocchio, The (1996)
974                                  Fly Away Home (1996)
1002        Homeward Bound: The Incredible Journey (1993)
1004                         Swiss Family Robinson (1960)
1500       Second Jungle Book: Mowgli & Baloo, The (1997)
1541                                  Wild America (1997)
1759          

***Checking whether are recommendations are correct or not by matching it with genres.***

In [65]:
df.loc[df['Title'].isin(['Tom and Huck (1995)','Amazing Panda Adventure, The (1995)','In Search of the Castaways (1962)'])]

,MovieID,Title,Genres,year
7,8,Tom and Huck (1995),Adventure Children's,1995
144,146,"Amazing Panda Adventure, The (1995)",Adventure Children's,1995
1987,2056,In Search of the Castaways (1962),Adventure Children's,1962
